In [57]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [58]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables

In [59]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame


Row,Hour,W1,W2,W3,W4,W5,W6
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,76.8921,101.54,92.8003,95.3709,96.002,70.9073
2,2,66.8277,90.9989,109.167,107.937,103.616,112.618
3,3,78.422,116.959,142.88,134.78,128.276,133.832
4,4,64.1437,134.249,159.57,133.664,142.969,154.233
5,5,102.22,145.507,160.944,165.303,144.03,161.49
6,6,134.039,131.112,157.023,161.828,144.636,155.222
7,7,146.517,153.67,141.566,159.927,152.86,153.604
8,8,143.176,163.392,155.294,170.805,158.643,164.307
9,9,163.297,144.48,172.638,185.318,150.901,166.558


In [60]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

4

In [61]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])
equilibrium_df = DataFrame(hour = Int[], market_price = Float64[], market_quantity = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

In [62]:

hour = 18 
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator
unregister(model, :x)  # Unregister the existing variable named "x" from the model

#demand bids
name = "demand_hour_" * string(hour-1) * ".csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame

# number of demand
D = size(demand_bids, 1)

#@variable(model, x_bin[1:G], Bin)  # binary variable for on/off status
@variable(model, x_con[1:G])  # power output variable

# wind decision variables
@variable(model, x_wind[1:W])  # Create the new variable

# Create explicit constraints for the bounds, to be able to access their dual values later
lb_g = Vector{Any}(undef, G+W)
ub_g = Vector{Any}(undef, G+W)
for g in 1:G
    lb_g[g] = @constraint(model, x_con[g] >= 0)
    ub_g[g] = @constraint(model, x_con[g] <= con_generation[g,6])
end
for w in 1:W
    lb_g[G+w] = @constraint(model, x_wind[w] >= 0)
    ub_g[G+w] = @constraint(model, x_wind[w] <= wind_profile[hour,w+1])
end



# decision variables for demand
@variable(model, y[1:size(demand_bids, 1)])  # Create the new variable
# Create explicit constraints for the bounds, to be able to access their dual values later
lb_d = Vector{Any}(undef, D)
ub_d = Vector{Any}(undef, D)
for d in 1:D
    lb_d[d] = @constraint(model, y[d] >= 0)
    ub_d[d] = @constraint(model, y[d] <= demand_bids[d,2])
end

# Add the constraint for the balance between supply and demand
@constraint(model, BalanceConstraint,  sum(x_con[g] for g in 1:G) + sum(x_wind[w] for w in 1:W) - sum(y[d] for d in 1:D) == 0)

# Define the objective function
@objective(model, Max, sum(demand_bids[d,3] * y[d] for d in 1:D) - sum(con_generation[g,3] * x_con[g] for g in 1:G) - sum(0.0* x_wind[w] for w in 1:W))

# Print the model
#print(model)

# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Hour ", hour, ": Optimal solution found")
    #println("Objective value: ", objective_value(model))
    #println("x values: ", [value(x[i]) for i in 1:G])
    #println("x_wind values: ", [value(x_wind[i]) for i in 1:W])
    #println("y values: ", [value(y[i]) for i in 1:D])
    println(dual(BalanceConstraint))
    #add dual values to dataframe
    push!(equilibrium_df, [hour, dual(BalanceConstraint), sum(value(y[i]) for i in 1:D)])

    #save results to dataframe
    push!(result_df, [hour, objective_value(model), [value(x_con[i]) for i in 1:G]..., [value(x_wind[i]) for i in 1:W]..., [value(y[i]) for i in 1:D]...])

elseif status == MOI.INFEASIBLE
    println("Hour ", hour, ": No feasible solution found")
else
    println("Hour ", hour, ": Solver status: ", status)
end


Hour 18: Optimal solution found
13.32


Row,hour,objective_value,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,18,35838.0,21.9257,0.0,0.0,0.0,0.0,155.0,155.0,400.0,400.0,300.0,310.0,350.0,131.297,145.216,134.656,147.406,100.719,90.117,246.497,180.234,294.205,92.7675,310.108,169.632,119.273,166.981,68.913,66.2625,127.224,116.622,159.03,161.68,180.234


In [63]:
#Access the dual values of all the generators constraints and add them to a new dataframe, in scientific notation
duals_g = DataFrame(hour = Int[], gen = Symbol[], lower_bound = Float64[], upper_bound = Float64[])
for i in 1:G
    push!(duals_g, [hour, Symbol("x_conv$i"), dual(lb_g[i]), dual(ub_g[i])])
end
for i in 1:W
    push!(duals_g, [hour, Symbol("x_wind$i"), dual(lb_g[i+G]), dual(ub_g[i+G])])
end

duals_d = DataFrame(hour = Int[], node = Int[],lower_bound = Float64[], upper_bound = Float64[])
for i in 1:D
    push!(duals_d, [hour, demand_bids[i,1], dual(lb_d[i]), dual(ub_d[i])])
end


#save results to dataframe
CSV.write("results/optimization_results_generation.csv", duals_g)
CSV.write("results/optimization_results_demand.csv", duals_d)


"results/optimization_results_demand.csv"

In [64]:
result_df

Row,hour,objective_value,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,18,35838.0,21.9257,0.0,0.0,0.0,0.0,155.0,155.0,400.0,400.0,300.0,310.0,350.0,131.297,145.216,134.656,147.406,100.719,90.117,246.497,180.234,294.205,92.7675,310.108,169.632,119.273,166.981,68.913,66.2625,127.224,116.622,159.03,161.68,180.234


In [65]:
#prepare the plot
demand_sort = DataFrame(Hour = Int[], Quantity_MW = Float64[], Price_MWh = Float64[])
for d in 1:D
    push!(demand_sort, (Hour = hour, Quantity_MW = 0, Price_MWh = demand_bids[d, 3]))
    push!(demand_sort, (Hour = hour, Quantity_MW = demand_bids[d, 2], Price_MWh = demand_bids[d, 3]))
end
sort!(demand_sort, :Price_MWh, rev=true)

supply_sort = DataFrame(Hour = Int[], Quantity_MW = Float64[], Price_MWh = Float64[])
for g in 1:G
    push!(supply_sort, (Hour = hour, Quantity_MW = 0, Price_MWh = con_generation[g,3]))
    push!(supply_sort, (Hour = hour, Quantity_MW = con_generation[g,6], Price_MWh = con_generation[g,3]))
end
for w in 1:W
    push!(supply_sort, (Hour = hour, Quantity_MW = 0, Price_MWh = 0))
    push!(supply_sort, (Hour = hour, Quantity_MW = wind_profile[hour,w+1], Price_MWh = 0))
end

sort!(supply_sort, :Price_MWh)

for d in 2:2*D
    demand_sort[d,2] += demand_sort[d-1,2]
end
for s in 2:size(supply_sort,1)
    supply_sort[s,2] += supply_sort[s-1,2]
end

demand_sort
supply_sort

Row,Hour,Quantity_MW,Price_MWh
,Int64,Float64,Float64
1,18,0.0,0.0
2,18,131.297,0.0
3,18,131.297,0.0
4,18,276.512,0.0
5,18,276.512,0.0
6,18,411.168,0.0
7,18,411.168,0.0
8,18,558.574,0.0
9,18,558.574,5.47


In [66]:
using Plots
plot(demand_sort.Quantity_MW, demand_sort.Price_MWh, label="Demand", xlabel="Quantity_MW", ylabel="Price_€/MW",  lw=2)
plot!(supply_sort.Quantity_MW, supply_sort.Price_MWh, label="Supply", lw=2)
hline!([equilibrium_df[1, 2]], color="black", label="Market price")
vline!([equilibrium_df[1, 3]], label="Market quantity")

#title="Market Clearing",
#save plot in results folder
savefig("results/market_clearing_hour_" * string(hour) * ".png")

"c:\\Users\\janle\\Desktop\\Master_local\\2\\renewables_in_electricity_markets\\Task 1\\results\\market_clearing_hour_18.png"

In [67]:
#save result_df to csv
CSV.write("results/market_clearing_model1.csv", result_df)
CSV.write("results/equilibrium_model1.csv", equilibrium_df)

"results/equilibrium_model1.csv"

In [68]:

profit_df = DataFrame()
#make for each generator a column
for i in 1:G
    profit_df[!, Symbol("x_con$i")] = Float64[]
end

# insert the assigned capacity for each generator
capacities = []
for i in 1:G
    # make array with the assigned capacity
    push!(capacities, value(x_con[i]))
end

push!(profit_df, capacities)


#calculate the profit for each generator
profits = []
for i in 1:G
    push!(profits, value(x_con[i])* dual(BalanceConstraint)-con_generation[i,3]*value(x_con[i]))
end

push!(profit_df, profits)

#add dual values to dataframe
lwr_duals = []
upr_duals = []
for i in 1:G
    push!(lwr_duals, dual(lb_g[i]))
    push!(upr_duals, dual(ub_g[i]))
end
push!(profit_df, lwr_duals)
push!(profit_df, upr_duals)

CSV.write("results/profits_model1.csv", profit_df)

pretty_table(profit_df, backend = Val(:latex))



\begin{tabular}{rrrrrrrrrrrr}
  \hline
  \textbf{x\_con1} & \textbf{x\_con2} & \textbf{x\_con3} & \textbf{x\_con4} & \textbf{x\_con5} & \textbf{x\_con6} & \textbf{x\_con7} & \textbf{x\_con8} & \textbf{x\_con9} & \textbf{x\_con10} & \textbf{x\_con11} & \textbf{x\_con12} \\
  \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} \\\hline
  21.9257 & 0.0 & 0.0 & 0.0 & 0.0 & 155.0 & 155.0 & 400.0 & 400.0 & 300.0 & 310.0 & 350.0 \\
  0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 434.0 & 434.0 & 2920.0 & 3140.0 & 1896.0 & 868.0 & 850.5 \\
  -0.0 & -0.0 & 7.38 & 7.61 & 12.79 & -0.0 & -0.0 & -0.0 & -0.0 & -0.0 & -0.0 & -0.0 \\
  -0.0 & -0.0 & -0.0 & -0.0 & -0.0 & -2.8 & -2.8 & -7.3 & -7.85 & -6.32 & -2.8 & -2.43 \\\hline
\end{tabular}


In [69]:
#utility table for demand
utility_df = DataFrame()

#make for each demand a column
for i in 1:D
    utility_df[!, Symbol("y$i")] = Float64[]
end

# insert the assigned capacity for each demand
capacities = []
for i in 1:D
    # make array with the assigned capacity
    push!(capacities, value(y[i]))
end

push!(utility_df, capacities)


#calculate the utility for each demand
utilities = []
for i in 1:D
    push!(utilities, (demand_bids[i,3]-dual(BalanceConstraint))*value(y[i]))
end

push!(utility_df, utilities)

#add dual values to dataframe
lwr_duals = []
upr_duals = []
for i in 1:D
    push!(lwr_duals, dual(lb_d[i]))
    push!(upr_duals, dual(ub_d[i]))
end
push!(utility_df, lwr_duals)
push!(utility_df, upr_duals)

CSV.write("results/utilities_model1.csv", utility_df)

pretty_table(utility_df, backend = Val(:latex))

In [70]:
println(demand_bids)

\begin{tabular}{rrrrrrrrrrrrrrrrr}
  \hline
  \textbf{y1} & \textbf{y2} & \textbf{y3} & \textbf{y4} & \textbf{y5} & \textbf{y6} & \textbf{y7} & \textbf{y8} & \textbf{y9} & \textbf{y10} & \textbf{y11} & \textbf{y12} & \textbf{y13} & \textbf{y14} & \textbf{y15} & \textbf{y16} & \textbf{y17} \\
  \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} & \texttt{Float64} \\\hline
  100.719 & 90.117 & 246.497 & 180.234 & 294.205 & 92.7675 & 310.108 & 169.632 & 119.273 & 166.981 & 68.913 & 66.2625 & 127.224 & 116.622 & 159.03 & 161.68 & 180.234 \\
  805.071 & 588.067 & 1540.6 & 1380.57 & 2034.12 & 577.564 & 2038.23 & 1094.81 & 918.724 & 1193.62 & 586.198 & 537.032 & 641.639 & 830.041 & 943.387 & 967.485 & 1178.16 \\
  -0.0 & -0.0 & -0.0 & -0.0 & -0.0